# DREAM dataset

In [1]:
import pandas as pd
import numpy as np
import glob
import json

In [2]:
def item_or_default(array, index, default_value=None):
    try:
        return array[index]
    except IndexError:
        return default_value

def convert_json_to_dataframe(filename):
    with open(filename) as json_file:
        contents = json.load(json_file)

        pre_test = {f'pre_test_{k}': v for k, v in contents.get('ados', {}).get('preTest', {}).items()}
        participant = contents.get('participant')
        frame_rate = contents.get('frame_rate')
        condition = contents.get('condition')
        task = {f'task_{k}': v for k, v in contents.get('task', {}).items()}
        eye_gaze = contents.get('eye_gaze')
        head_gaze = contents.get('head_gaze')

        metadata = {**pre_test, **participant, 'frame_rate': frame_rate, 'condition': condition, **task}

        length = max(len(eye_gaze['rx']), len(eye_gaze['ry']), len(eye_gaze['rz']), len(head_gaze['rx']), len(head_gaze['ry']), len(head_gaze['rz']))

        current_data = []
        for i in range(0, length):
            current_data.append({
                **metadata,
                'gaze_index': i,
                'eye_gaze_rx': item_or_default(eye_gaze['rx'], i),
                'eye_gaze_ry': item_or_default(eye_gaze['ry'], i),
                'eye_gaze_rz': item_or_default(eye_gaze['rz'], i),
                'head_gaze_rx': item_or_default(head_gaze['rx'], i),
                'head_gaze_ry': item_or_default(head_gaze['ry'], i),
                'head_gaze_rz': item_or_default(head_gaze['rz'], i)
            })

        return pd.DataFrame(current_data)
    

In [4]:
dream_df = None

files = glob.glob('./assets/DREAMdataset/**/*.json', recursive=True)

for filename in files:
    df = convert_json_to_dataframe(filename)
    
    if dream_df is not None:
        dream_df = pd.concat([dream_df, df])
    else:
        dream_df = df

In [100]:
# dream_df['task_difficultyLevel'].unique()
dream_df.head()

,pre_test_communication,pre_test_interaction,pre_test_module,pre_test_play,pre_test_protocol,pre_test_socialCommunicationQuestionnaire,pre_test_stereotype,pre_test_total,ageInMonths,gender,...,task_end,task_index,task_start,gaze_index,eye_gaze_rx,eye_gaze_ry,eye_gaze_rz,head_gaze_rx,head_gaze_ry,head_gaze_rz
0,5,10,1.0,2,ADOS-G,15.0,4,15,67,female,...,2977,64,0,0,-0.537899,-0.092985,-0.83596,-0.686629,-0.051143,-0.723528
1,5,10,1.0,2,ADOS-G,15.0,4,15,67,female,...,2977,64,0,1,NaN,NaN,NaN,NaN,NaN,NaN
2,5,10,1.0,2,ADOS-G,15.0,4,15,67,female,...,2977,64,0,2,NaN,NaN,NaN,NaN,NaN,NaN
3,5,10,1.0,2,ADOS-G,15.0,4,15,67,female,...,2977,64,0,3,NaN,NaN,NaN,NaN,NaN,NaN
4,5,10,1.0,2,ADOS-G,15.0,4,15,67,female,...,2977,64,0,4,NaN,NaN,NaN,NaN,NaN,NaN
